In [1]:
import os
import json
import pandas as pd
import datetime
from google.cloud import storage

In [2]:
# connect to gcloud bucket
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'survai-data-connect.json'
storage_client = storage.Client()
bucket_name = 'survai-dataset'
bucket = storage_client.bucket(bucket_name)

In [12]:
# access temp_videodata_storage folder
blobs = storage_client.list_blobs(bucket_name, prefix="datasets/test_datasets/sparverius_pf/results_08052022/temp_videodata_storage/")
data = []

for blob in blobs:

    file = blob.name.split("/")[5]
    i = file.count("_")

    if i > 1:
        file_name = "_".join(file.split("_", i)[:i]) 
    else:
        file_name = file.split("_")[0]

    if "od.json" in file:
        od = json.loads(blob.download_as_string())
    
        # change seconds to datetime
        for k,v in list(od.items()):
            newkey = str(datetime.timedelta(seconds=int(k)))
            od[newkey] = od.pop(k)
        
        data.append({
            "file_name": f"{file_name}.mp4",
            "detections_per_second": od
        })

    if "stats.json" in file:
        stats = json.loads(blob.download_as_string())
        
        for k,v in stats.items():
            data.append({
            "file_name": f"{file_name}.mp4",
            k: v
        })

In [13]:
df = pd.DataFrame(data)
df = df.drop(['person_on_ground', 'spray', 'violent_actions', 'crowd', 
                'striking', 'running', 'throwing', 'brawling', 'arresting'], axis=1)
aggregate_functions = {'detections_per_second': 'first', 'Non Uniformed': 'sum', 
                        'Uniformed': 'sum', 'Riot Shield': 'sum', 'Gun': 'sum', 'Pepper Spray': 'sum',
                        'Baton': 'sum', 'Chemical Smoke': 'sum'}

df_new = df.groupby('file_name', as_index=False).aggregate(aggregate_functions).reindex(columns=df.columns)

In [14]:
#df_new.to_json('pf_detections.json', orient='index')
df_new.to_csv('pf_detections2.csv')